In [1]:
print("OK!")

OK!


In [1]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers


c:\Users\Abhishek SHUKLA\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
PINECONE_API_KEY = "2bd5fd9f-2c56-42f9-aa1b-037960262fff"
PINECONE_API_ENV = "gcp-starter"

In [3]:
#Extract data from the PDF
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()

    return documents

In [5]:
extracted_data = load_pdf("C:\\Users\\Abhishek SHUKLA\\OneDrive\\Documents\\medibot\\End-to-end-Medical-Chatbot-using-Llama2-main\\data")

In [7]:
# extracted_data

In [6]:
#Create text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [7]:
text_chunks = text_split(extracted_data)
print("length of my chunk:", len(text_chunks))

length of my chunk: 5860


In [11]:
# text_chunks

In [8]:
#download embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [2]:
embeddings = download_hugging_face_embeddings()

NameError: name 'download_hugging_face_embeddings' is not defined

In [1]:
embeddings

NameError: name 'embeddings' is not defined

In [11]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [17]:
# query_result

In [15]:
from dotenv import load_dotenv
load_dotenv()

True

In [17]:
#Initializing the Pinecone
import os
from pinecone import Pinecone,ServerlessSpec

pc=Pinecone(api_key="pcsk_EC9bZ_4bYKxBvtNskx9PC7xw8HkyfThrBVm7rbguQgAsKcm7dKYzKhXHgAif6vNPvyFkS")

index_name = "medicalbot"

pc.create_index(
    name=index_name,
    dimension=384, # Replace with your model dimensions
    metric="cosine", # Replace with your model metric
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    ) 
)

{
    "name": "medicalbot",
    "metric": "cosine",
    "host": "medicalbot-u7c4zkn.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 384,
    "deletion_protection": "disabled",
    "tags": null
}

In [22]:
from langchain_pinecone import PineconeVectorStore
docsearch=PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings,
)
retriever=docsearch.as_retriever(search_type="similarity",search_kwargs={"k":3})
retrieved_docs=retriever.invoke("What is acne?")

In [23]:
retrieved_docs

[]

In [24]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [25]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [26]:
llm=CTransformers(model="C:\\Users\\Abhishek SHUKLA\\OneDrive\\Documents\\medibot\\End-to-end-Medical-Chatbot-using-Llama2-main\\model",
                  model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature':0.8})

In [27]:
qa=RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=docsearch.as_retriever(search_kwargs={'k': 2}),
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs)

In [28]:
while True:
    user_input=input(f"Input Prompt:")
    result=qa({"query": user_input})
    print("Response : ", result["result"])

C:\Users\Abhishek SHUKLA\AppData\Local\Temp\ipykernel_20724\4216317267.py:3: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result=qa({"query": user_input})


Response :  Acne is a common skin condition characterized by redness, inflammation, and pimples on the face, chest, and back. It occurs when the pores on the skin become clogged with dead skin cells, oil, and bacteria, leading to infections and irritation. Acne can be caused by a variety of factors, including hormonal changes, genetics, and environmental factors such as stress and humidity. Treatment options for acne include topical creams and gels, oral antibiotics, and lifestyle changes such as regular exercise and a healthy diet.
Response :  "Sure! Here are some tips for getting your dog to stop barking:
1. Identify the reason for your dog's barking: Is it due to boredom, anxiety, or something else? Once you know the cause, you can take steps to address it.
2. Provide exercise and mental stimulation: Dogs need physical and mental stimulation to stay happy and healthy. Take your dog for a walk, play fetch, or engage in other activities that challenge their mind and body.
3. Train you

KeyboardInterrupt: 